In [ ]:
# consider the models...
# I believe the logistic regression model will out perform the random forest due to the high number of features in the dataset.



In [ ]:
# Considering the Models -

# After looking at the dataset and considering the two model types, I believe that the logistic regression model 
# will perform better for this dataset.  Both Random Forest and Logistic Regression models are good for classification 
# problems, but I believe with over 80 variables to consider, the Logistic model will win out.  Based on my research, 
# Random Forest is recommended for simpler classification problems.  

# Evaluating the Models - without scaling the data, the Random Forest model performed better on the test data.  Perhaps my understanding 
# of what a "simple" classification problem is is lacking, or the lack of scaling the data influenced the results.  

# Results w/ Scaling - Once I scaled the data, the logistic regression became the better classifier. It had an accuracy score of .66.  
# The Random Forest model decreased in accuracy from .64 to .5.  From what I understand, this is to be expected as Random Forest modeling
# doesn't rely on scaled data to make accurate calculations, while logistic regression models do.  

# While I was able to produce a model with a .66 accuracy metric, I would be concerned about employing this out into the wild for credit 
# worthiness rating purposes.  There is still far too much inaccuracy, in my opinion, and money would be lost on a consistent basis if 
# loan decisions were based on this model. 

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [3]:
test_df.columns

Index(['Unnamed: 0', 'index', 'loan_amnt', 'int_rate', 'installment',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_ol

In [4]:
target = "loan_status"

low_risk_rows = train_df[train_df[target] == 'low_risk']
high_risk_rows = train_df[train_df[target] == 'high_risk']

df = pd.concat([low_risk_rows, high_risk_rows.sample(n=len(low_risk_rows), replace=True)])
df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=42), high_risk_rows])
df = df.reset_index(drop=True)
train_df = df.rename({target:'target'}, axis="columns")
train_df

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,target,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57334,57334,12500.0,0.1033,405.28,MORTGAGE,140000.0,Verified,low_risk,n,...,100.0,33.3,0.0,0.0,285413.0,98496.0,9800.0,117693.0,N,N
1,113084,113084,11600.0,0.1171,383.69,ANY,85000.0,Not Verified,low_risk,n,...,100.0,20.0,0.0,0.0,49395.0,32657.0,10800.0,31095.0,N,N
2,215254,215254,15950.0,0.1308,363.57,MORTGAGE,26000.0,Verified,low_risk,n,...,100.0,0.0,0.0,0.0,237321.0,44402.0,24400.0,34596.0,N,N
3,49995,49995,25000.0,0.1894,915.65,OWN,168400.0,Source Verified,low_risk,n,...,100.0,28.6,0.0,0.0,128508.0,84853.0,98400.0,30108.0,N,N
4,315024,315024,10625.0,0.1695,378.55,MORTGAGE,31000.0,Verified,low_risk,n,...,87.9,33.3,0.0,0.0,47110.0,16209.0,9800.0,26410.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,354912,354912,19975.0,0.2565,801.09,RENT,28000.0,Not Verified,high_risk,n,...,100.0,16.7,0.0,0.0,50055.0,28192.0,18700.0,19055.0,N,N
12176,354944,354944,15000.0,0.1774,540.34,RENT,50000.0,Verified,high_risk,n,...,90.5,11.1,0.0,0.0,70324.0,57025.0,13300.0,54824.0,N,N
12177,354973,354973,3600.0,0.1862,131.28,RENT,60000.0,Not Verified,high_risk,n,...,100.0,0.0,0.0,0.0,83765.0,55156.0,14800.0,53065.0,N,N
12178,355002,355002,15000.0,0.0881,475.68,MORTGAGE,62000.0,Source Verified,high_risk,n,...,100.0,0.0,0.0,0.0,189930.0,23748.0,7000.0,32930.0,N,N


In [5]:
# Convert categorical data to numeric and separate target feature for training data

X_train = train_df.drop('target', axis=1) 

X_dummies_train = pd.get_dummies(X_train)
y_train = train_df['target']

X_dummies_train

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,57334,57334,12500.0,0.1033,405.28,140000.0,18.28,0.0,0.0,12.0,...,1,1,0,1,1,0,1,0,1,0
1,113084,113084,11600.0,0.1171,383.69,85000.0,13.77,0.0,0.0,11.0,...,0,1,0,1,1,0,1,0,1,0
2,215254,215254,15950.0,0.1308,363.57,26000.0,55.25,0.0,1.0,14.0,...,1,1,0,1,0,1,1,0,1,0
3,49995,49995,25000.0,0.1894,915.65,168400.0,5.91,0.0,1.0,8.0,...,0,1,0,1,1,0,1,0,1,0
4,315024,315024,10625.0,0.1695,378.55,31000.0,32.44,0.0,1.0,13.0,...,1,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,354912,354912,19975.0,0.2565,801.09,28000.0,28.42,0.0,0.0,15.0,...,0,1,0,1,1,0,1,0,1,0
12176,354944,354944,15000.0,0.1774,540.34,50000.0,23.43,4.0,0.0,16.0,...,1,1,0,1,1,0,1,0,1,0
12177,354973,354973,3600.0,0.1862,131.28,60000.0,28.80,0.0,1.0,14.0,...,0,1,0,1,1,0,1,0,1,0
12178,355002,355002,15000.0,0.0881,475.68,62000.0,11.44,0.0,0.0,5.0,...,0,1,0,1,0,1,1,0,1,0


In [6]:
# Convert categorical data to numeric and separate target feature for testing data

X_test = test_df.drop('loan_status', axis=1) 

X_dummies_test = pd.get_dummies(X_test)
y_test = test_df['loan_status']

X_dummies_test

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
0,67991,67991,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,...,0,0,1,0,1,1,0,1,0,1
1,25429,25429,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,...,0,0,1,0,1,1,0,1,0,1
2,38496,38496,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,...,0,0,1,0,1,1,0,1,0,1
3,19667,19667,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,...,0,0,1,0,1,1,0,1,0,1
4,37505,37505,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,...,0,0,1,0,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,77282,77282,30000.0,0.1240,673.42,140480.0,15.74,0.0,0.0,20.0,...,1,0,1,1,0,1,0,1,0,1
4698,77291,77291,24000.0,0.0756,747.22,50000.0,26.81,0.0,0.0,9.0,...,0,0,1,0,1,1,0,1,0,1
4699,77292,77292,10000.0,0.2305,387.36,33000.0,38.51,0.0,2.0,7.0,...,0,1,1,1,0,1,0,1,0,1
4700,77297,77297,8000.0,0.1862,205.86,38000.0,16.36,0.0,1.0,8.0,...,1,0,1,0,1,1,0,1,0,1


In [7]:
# add missing dummy variables to testing set

for c in X_dummies_train.columns:
    if c not in X_dummies_test.columns:
        X_dummies_test[c] = 0

In [8]:
# Train the Logistic Regression model on the unscaled data and print the model score

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='lbfgs',max_iter=15000)
classifier.fit(X_dummies_train, y_train)

print(f'Training Data Score: {classifier.score(X_dummies_train, y_train)}')
print(f'Testing Data Score: {classifier.score(X_dummies_test, y_test)}')

Training Data Score: 0.7027093596059113
Testing Data Score: 0.5655040408336878


In [9]:
# Train a Random Forest Classifier model and print the model score

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=1, n_estimators=50).fit(X_dummies_train, y_train)

print(f'Training Score: {clf.score(X_dummies_train, y_train)}')
print(f'Testing Score: {clf.score(X_dummies_test, y_test)}')

Training Score: 0.9999178981937603
Testing Score: 0.633134836239898


In [ ]:
# thru the first test the Random Forest Classifier was better - I was wrong.

In [10]:
# Scale the data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_dummies_train)

X_train_scaled = scaler.transform(X_dummies_train)
X_test_scaled = scaler.transform(X_dummies_test)

In [11]:
# Train the Logistic Regression model on the scaled data and print the model score

classifier.fit(X_train_scaled, y_train)

print(f'Training Data Score: {classifier.score(X_train_scaled, y_train)}')
print(f'Testing Data Score: {classifier.score(X_test_scaled, y_test)}')

Training Data Score: 0.7126436781609196
Testing Data Score: 0.7203317737133135


In [12]:
# Train a Random Forest Classifier model on the scaled data and print the model score

clf.fit(X_train_scaled, y_train)

print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

Testing Score: 0.6320714589536367


In [ ]:
# After scaling the logistic regression model turned out to be the better choice here. With that said, neither would model would work for actually predicting credit worthiness in a real world setting.